In [4]:
import pandas as pd

In [6]:
url="http://stat.kg/ru/opendata/category/4925/json"
# Read JSON data and create a DataFrame
data = pd.read_json(url)
data

,data,title,units
0,"{'pk': 120005, 'priority': 1, 'title_en': 'Emp...",Уровень занятости - женщины Кыргызская Республика,
1,"{'pk': 120006, 'priority': 2, 'title_en': 'Bat...",Уровень занятости - женщины Кыргызская Республика,
2,"{'pk': 120007, 'priority': 3, 'title_en': 'Dja...",Уровень занятости - женщины Кыргызская Республика,
3,"{'pk': 120008, 'priority': 4, 'title_en': 'Iss...",Уровень занятости - женщины Кыргызская Республика,
4,"{'pk': 120009, 'priority': 5, 'title_en': 'Nar...",Уровень занятости - женщины Кыргызская Республика,
5,"{'pk': 120010, 'priority': 6, 'title_en': 'Osh...",Уровень занятости - женщины Кыргызская Республика,
6,"{'pk': 120011, 'priority': 7, 'title_en': 'Tal...",Уровень занятости - женщины Кыргызская Республика,
7,"{'pk': 120012, 'priority': 8, 'title_en': 'Chu...",Уровень занятости - женщины Кыргызская Республика,
8,"{'pk': 120013, 'priority': 9, 'title_en': 'Bis...",Уровень занятости - женщины Кыргызская Республика,
9,"{'pk': 120014, 'priority': 10, 'title_en': 'Os...",Уровень занятости - женщины Кыргызская Республика,
